In [ ]:
import numpy as np
import scipy as sp
import scipy.sparse as sparse
import matplotlib.pyplot as plt
%matplotlib inline

np.set_printoptions(precision=4,suppress=True)

from dedalus_sphere import jacobi as Jacobi

from spherinder import config
config.internal_dtype = 'float128'

import spherinder.operators as sph
from spherinder.operators import Basis, plotfield

In [ ]:
ns, neta = 500, 501
s, eta = np.linspace(0,1,ns), np.linspace(-1,1,neta)

g_disable_basis_plots = False

In [ ]:
def plot_basis(basis, ell, s, eta):
    Nmax = basis.Nmax
    ncols = 3
    nrows = (Nmax+ncols-1)//ncols
    fig, plot_axes = plt.subplots(nrows,ncols,figsize=(11,4*nrows))
    row, col = 0, 0
    for k in range(Nmax):
        plotfield(s, eta, basis[ell,k], fig, plot_axes[row][col])
        col += 1
        if col == ncols:
            col = 0
            row += 1


def plot_radial(basis, ell, s, eta, etavalues):
    if np.isscalar(etavalues): etavalues = [etavalues]
    neta = len(etavalues)

    fig, plot_axes = plt.subplots(1,neta,figsize=(4*neta,4))
    if neta == 1: plot_axes = [plot_axes]
    for i in range(neta):
        etaindex = np.argmin(np.abs(eta-etavalues[i]))
        for k in range(basis.Nmax):
            plot_axes[i].plot(s,basis[ell,k][etaindex,:])
        plot_axes[i].grid()
        plot_axes[i].set_title('ell = {}, η = {}'.format(ell, eta[etaindex]))
        plot_axes[i].set_xlabel('s')
        plot_axes[i].grid(True)


In [ ]:
m, Lmax, Nmax = 1, 13, 6
basis = Basis(s, eta, m, Lmax, Nmax, sigma=0, alpha=0, galerkin=True)

In [ ]:
ell = 0

if not g_disable_basis_plots:
    plot_basis(basis, ell, s, eta)
    plot_radial(basis, ell, s, eta, [0.0, 0.5, 1.0])

In [ ]:
ell = 1

if not g_disable_basis_plots:
    plot_basis(basis, ell, s, eta)
    plot_radial(basis, ell, s, eta, [0.0, 0.5, 1.0])

In [ ]:
ell = 2

if not g_disable_basis_plots:
    plot_basis(basis, ell, s, eta)
    plot_radial(basis, ell, s, eta, [0.0, 0.5, 1.0])

In [ ]:
ell = 3

if not g_disable_basis_plots:
    plot_basis(basis, ell, s, eta)
    plot_radial(basis, ell, s, eta, [0.0, 0.5, 1.0])

In [ ]:
ell = 4

if not g_disable_basis_plots:
    plot_basis(basis, ell, s, eta)
    plot_radial(basis, ell, s, eta, [0.0, 0.5, 1.0])

In [ ]:
ell = 5

if not g_disable_basis_plots:
    plot_basis(basis, ell, s, eta)
    plot_radial(basis, ell, s, eta, [0.0, 0.5, 1.0])

In [ ]:
ell = 12

if not g_disable_basis_plots:
    plot_basis(basis, ell, s, eta)
    plot_radial(basis, ell, s, eta, [0.0, 0.5, 1.0])

In [ ]:
A = Jacobi.operator('A')
B = Jacobi.operator('B')

In [ ]:
Lmax, alpha = 8, 0

bc = Jacobi.polynomials(Lmax,alpha,alpha,1.0)
a_ell_0 = bc[2:]/bc[:-2]

bc = Jacobi.polynomials(Lmax,alpha+1,alpha+1,1.0)
a_ell_1 = bc[2:]/bc[:-2]

eta = np.linspace(-1.,1.,100)
Peta = Jacobi.polynomials(Lmax,alpha,alpha,eta).T
peta0 = Peta[:,2:] - a_ell_0 * Peta[:,:-2]
Peta = Jacobi.polynomials(Lmax,alpha+1,alpha+1,eta).T
peta1 = Peta[:,2:] - a_ell_1 * Peta[:,:-2]

fig, ax = plt.subplots(1,2,figsize=(9,4))
ax[0].plot(eta,peta0)
ax[1].plot(eta,peta1)

AB = (A(+1) @ B(+1))(Lmax,alpha,alpha).todense()
alpha_ell = np.diag(AB)

ratio = alpha_ell[:-2]/alpha_ell[2:] * a_ell_0
debug = lambda name: print(name + ' = ' + repr(eval(name)))
debug('a_ell_0')
debug('a_ell_1')
debug('ratio')
debug('ratio / a_ell_1')

In [ ]:
m, Lmax, Nmax, alpha, sigma = 1, 5, 5, 1, 0
Op = sph.operator('1-r**2')(m, Lmax, Nmax, alpha, sigma)

print(np.shape(Op))
print(np.shape(sph.remove_zero_rows(Op)))

In [ ]:
ABd = (A(-1) @ B(-1))(Lmax,alpha,alpha).todense()
zmat = np.diag(np.diag(ABd,-2),-2)[:,:Lmax]
print(ABd)
print(zmat)
